In [1]:
import pyodbc
import pandas as pd
from pathlib import Path
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from datetime import date
import numpy as np
import getpass
import time

# for collaboration
username = getpass.getuser()
username

# Start the timer
start_time = time.time()

In [2]:
# home and time
home = Path.home()
todaystr = date.today().strftime('%Y-%m-%d')
# PNFV_alternative = pd.read_excel(Path(home, 'HP Inc', 'GPS TW Innovation - Documents', 'Users', 'GPS', 'PNFV', 'alternative.xlsx'))

if username == 'panj':
    PNFV = pd.read_excel(Path(home, 'HP Inc', 'GPS TW Innovation - 文件', 'Users', 'GPS', 'Shortage management related (Ri Xin)','PN FV description mapping table_ALL.xlsx')) # Jesse    
else:    
    PNFV = pd.read_excel(Path(home, 'HP Inc', 'GPS TW Innovation - Documents', 'Users', 'GPS', 'Shortage management related (Ri Xin)','PN FV description mapping table_ALL.xlsx')) # Dustin

# someone keep upload duplicated rows
PNFV = PNFV.drop_duplicates()
PNFV = PNFV[['Commodity','Supplier','PN','Descr','Alternative part flag']]

"""
currently the folder for buyer to update is still in GPSTW SOP - 2021 日新
REMEMBER to change the path when the folder officailly changed to GPS TW Innovation
"""

'\ncurrently the folder for buyer to update is still in GPSTW SOP - 2021 日新\nREMEMBER to change the path when the folder officailly changed to GPS TW Innovation\n'

In [3]:
# find the old part and upload new
conn = pyodbc.connect('Driver={SQL Server Native Client 11.0}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
cursor = conn.cursor()

# download from SQL
cursor.execute(f"SELECT * FROM OPS.GPS_tbl_ops_PN_FV")
PNFV_from_SQL = pd.DataFrame.from_records(cursor.fetchall(), columns = [i[0] for i in cursor.description])
PNFV_from_SQL['alternative part flag'] = PNFV_from_SQL['alternative part flag'].replace({None: np.nan,'nan':np.nan})
PNFV_from_SQL = PNFV_from_SQL.rename(columns={'alternative part flag':'Alternative part flag'})

# make adjustmnet 
PNFV_from_SQL['Alternative part flag'] = PNFV_from_SQL['Alternative part flag'].astype(str)
PNFV['Alternative part flag'] = PNFV['Alternative part flag'].astype(str)

# find the rows need to upload and delete
diff_df = PNFV_from_SQL.merge(PNFV, indicator=True, how='outer', on = ['Commodity','Supplier','PN','Descr','Alternative part flag'])
rows_to_delete = diff_df[diff_df['_merge'] == 'left_only']
rows_to_insert = diff_df[diff_df['_merge'] == 'right_only']

print(str(len(rows_to_delete)) + ' rows to delete')
print(str(len(rows_to_insert)) + ' rows to upload')

0 rows to delete
0 rows to upload


In [4]:
# Delete rows from the SQL table that are not in the local DataFrame
for index, row in rows_to_delete.iterrows():
    cursor.execute(f"DELETE FROM OPS.GPS_tbl_ops_PN_FV WHERE Commodity = ? AND Supplier = ? AND PN = ? AND Descr = ?", 
                   row['Commodity'], row['Supplier'], row['PN'], row['Descr'])
conn.commit()

# Upload the local file rows to SQL table
for index, row in rows_to_insert.iterrows():
    # Directly use None for null values
    commodity = row['Commodity'] if pd.notnull(row['Commodity']) else None
    supplier = row['Supplier'] if pd.notnull(row['Supplier']) else None
    pn = row['PN'] if pd.notnull(row['PN']) else None
    descr = row['Descr'] if pd.notnull(row['Descr']) else None
    alternative = row['Alternative part flag'] if pd.notnull(row['Alternative part flag']) else None

    # Execute the insert statement with parameters
    cursor.execute("INSERT INTO OPS.GPS_tbl_ops_PN_FV (Commodity, Supplier, PN, Descr, [Alternative part flag]) VALUES (?, ?, ?, ?, ?)", 
                commodity, supplier, pn, descr, alternative)
    conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

In [5]:
# check whether upload successfully
conn = pyodbc.connect('Driver={SQL Server Native Client 11.0}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
cursor = conn.cursor()

# download from SQL
cursor.execute(f"SELECT * FROM OPS.GPS_tbl_ops_PN_FV")
PNFV_from_SQL_new = pd.DataFrame.from_records(cursor.fetchall(), columns = [i[0] for i in cursor.description])

# check diff
diff_df_new = PNFV_from_SQL_new.merge(PNFV, indicator=True, how='outer', on = ['Commodity','Supplier','PN','Descr'])
rows_to_delete_new = diff_df_new[diff_df_new['_merge'] == 'left_only']
rows_to_insert_new = diff_df_new[diff_df_new['_merge'] == 'right_only']

# Check if there are no rows to delete or insert
if rows_to_delete_new.empty and rows_to_insert_new.empty:
    print('Records aligned, upload successfully!')
else:
    print('Please try again, something went wrong.')

Records aligned, upload successfully!


In [6]:
# End the timer
end_time = time.time()
print("The code used", round(end_time-start_time,2), "seconds, it is super fast!")

The code used 9.82 seconds, it is super fast!


In [7]:
# check the duplicated rows
duplicate_rows = PNFV_from_SQL[PNFV_from_SQL.duplicated()]
for index,row in duplicate_rows.iterrows():
    delete_query = "DELETE FROM CSI.OPS.GPS_tbl_ops_PN_FV WHERE Commodity =? AND Supplier=? AND PN=? AND Descr=? AND [Alternative part flag]=?"
    cursor.execute(delete_query,(row['Commodity'],row['Supplier'],row['PN'],row['Descr'],str(row['Alternative part flag'])))
    if cursor.rowcount:
        # print(row)
        print(f"{row['Commodity']},{row['Supplier']},{row['PN']},{row['Descr']} deleted from CSI.OPS.GPS_tbl_ops_PN_FV")
conn.commit()
for index, row in duplicate_rows.iterrows():
    s_Commodity = row['Commodity']
    s_supplier = row['Supplier']
    s_PN = row['PN']
    s_descr = row['Descr']
    s_alternative = str(row['Alternative part flag'])

    cursor.execute(f"INSERT INTO CSI.OPS.GPS_tbl_ops_PN_FV (Commodity, Supplier, PN, Descr, [Alternative part flag] )\
                    VALUES('{s_Commodity}','{s_supplier}','{s_PN}','{s_descr}','{s_alternative}')".replace("'NaN'", "'NULL'"))
    if cursor.rowcount:
        # print(row)
        print(f"{row['Commodity']},{row['Supplier']},{row['PN']},{row['Descr'] }uploaded to CSI.OPS.GPS_tbl_ops_PN_FV")
conn.commit()
conn.close()

CPU,INTEL,N25852-N08,DT INTEL Raptor Lake i7-13700 deleted from CSI.OPS.GPS_tbl_ops_PN_FV
Memory,MICRON,N20973-682,16GB DDR5-5600 deleted from CSI.OPS.GPS_tbl_ops_PN_FV
CPU,INTEL,N25852-N08,DT INTEL Raptor Lake i7-13700uploaded to CSI.OPS.GPS_tbl_ops_PN_FV
Memory,MICRON,N20973-682,16GB DDR5-5600uploaded to CSI.OPS.GPS_tbl_ops_PN_FV
CPU,INTEL,N25852-N08,DT INTEL Raptor Lake i7-13700uploaded to CSI.OPS.GPS_tbl_ops_PN_FV
CPU,INTEL,N25852-N08,DT INTEL Raptor Lake i7-13700uploaded to CSI.OPS.GPS_tbl_ops_PN_FV
Memory,MICRON,N20973-682,16GB DDR5-5600uploaded to CSI.OPS.GPS_tbl_ops_PN_FV
Memory,MICRON,N20973-682,16GB DDR5-5600uploaded to CSI.OPS.GPS_tbl_ops_PN_FV
CPU,INTEL,N25852-N08,DT INTEL Raptor Lake i7-13700uploaded to CSI.OPS.GPS_tbl_ops_PN_FV
CPU,INTEL,N25852-N08,DT INTEL Raptor Lake i7-13700uploaded to CSI.OPS.GPS_tbl_ops_PN_FV
Memory,MICRON,N20973-682,16GB DDR5-5600uploaded to CSI.OPS.GPS_tbl_ops_PN_FV
Memory,MICRON,N20973-682,16GB DDR5-5600uploaded to CSI.OPS.GPS_tbl_ops_PN_FV
CPU,IN